# MMDeploy部署MMClassification图像分类模型

推荐代码运行[云GPU环境](https://featurize.cn?s=d7ce99f842414bfcaea5662a97581bd1)：GPU RTX 3060、CUDA v11.2

OpenMMLab 主页：https://openmmlab.com/

MMDeploy主页：https://github.com/open-mmlab/mmdeploy

作者：同济子豪兄 2022-9-5

## 进入 mmclassification 目录

In [1]:
import os
os.chdir('mmclassification')

## 下载MMClassification训练好的模型config配置文件和权重文件

In [2]:
!wget https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20220716-mmclassification/configs/mobilenet_v2_1x_fruit30.py -P configs/mobilenet_v2

!wget https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20220716-mmclassification/checkpoints/fruit30_mmcls/fruit30_mmcls.pth -P checkpoint


--2022-09-05 23:05:49--  https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20220716-mmclassification/configs/mobilenet_v2_1x_fruit30.py
正在连接 172.16.0.13:5848... 已连接。
已发出 Proxy 请求，正在等待回应... 200 OK
长度： 1766 (1.7K) [binary/octet-stream]
正在保存至: “configs/mobilenet_v2/mobilenet_v2_1x_fruit30.py”

mobilenet_v2_1x_fru 100%[===================>]   1.72K  --.-KB/s    用时 0s      

2022-09-05 23:05:49 (41.7 MB/s) - 已保存 “configs/mobilenet_v2/mobilenet_v2_1x_fruit30.py” [1766/1766])

--2022-09-05 23:05:50--  https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20220716-mmclassification/checkpoints/fruit30_mmcls/fruit30_mmcls.pth
正在连接 172.16.0.13:5848... 已连接。
已发出 Proxy 请求，正在等待回应... 200 OK
长度： 27493269 (26M) [binary/octet-stream]
正在保存至: “checkpoint/fruit30_mmcls.pth”

fruit30_mmcls.pth   100%[===================>]  26.22M  16.6MB/s    用时 1.6s    

2022-09-05 23:05:51 (16.6 MB/s) - 已保存 “checkpoint/fruit30_mmcls.pth” [27493269/27493269])



## 返回上一级目录

In [3]:
%cd ..

/home/featurize/work/MMDeploy_Tutorials


## 模型转为ONNX中间格式

In [1]:
!python mmdeploy/tools/deploy.py \
        mmdeploy/configs/mmcls/classification_onnxruntime_dynamic.py \
        mmclassification/configs/mobilenet_v2/mobilenet_v2_1x_fruit30.py \
        mmclassification/checkpoint/fruit30_mmcls.pth \
        mmclassification/demo/demo.JPEG \
        --work-dir mmdeploy_models/mmcls/fruit30_mmcls \
        --device cpu \
        --dump-info


2022-09-05 23:26:36,392 - mmdeploy - INFO - Start pipeline mmdeploy.apis.pytorch2onnx.torch2onnx in subprocess
load checkpoint from local path: mmclassification/checkpoint/fruit30_mmcls.pth
2022-09-05 23:26:39,205 - mmdeploy - WARNING - DeprecationWarning: get_onnx_config will be deprecated in the future. 
2022-09-05 23:26:39,205 - mmdeploy - INFO - Export PyTorch model to ONNX: mmdeploy_models/mmcls/fruit30_mmcls/end2end.onnx.
2022-09-05 23:26:41,587 - mmdeploy - INFO - Execute onnx optimize passes.
2022-09-05 23:26:41,772 - mmdeploy - INFO - Finish pipeline mmdeploy.apis.pytorch2onnx.torch2onnx
2022-09-05 23:26:42,331 - mmdeploy - WARNING - "visualize_model" has been skipped may be because it's             running on a headless device.
2022-09-05 23:26:42,332 - mmdeploy - INFO - All process success.


## 推理预测

In [7]:
# 下载测试图片
!wget https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20220716-mmclassification/test/watermelon1.jpg


--2022-09-05 17:38:34--  https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20220716-mmclassification/test/watermelon1.jpg
正在连接 172.16.0.13:5848... 已连接。
已发出 Proxy 请求，正在等待回应... 200 OK
长度： 429666 (420K) [image/jpeg]
正在保存至: “watermelon1.jpg”

watermelon1.jpg     100%[===================>] 419.60K  1.96MB/s    用时 0.2s    

2022-09-05 17:38:44 (1.96 MB/s) - 已保存 “watermelon1.jpg” [429666/429666])



## 在命令行shell中运行

```shell
python mmdeploy/demo/python/image_classification.py --image_path watermelon1.jpg --model_path mmdeploy_models/mmcls/fruit30_mmcls/end2end.onnx --device_name cpu

python
```

```python
import cv2
from mmdeploy_python import Classifier
img = cv2.imread('watermelon1.jpg')
onnx_path = 'mmdeploy_models/mmcls/fruit30_mmcls/end2end.onnx'
classifier = Classifier(onnx_path, 'cpu')
result = classifier(img)
for label_id, score in result:
    print(label_id, score)
```

In [ ]:
python mmdeploy/demo/python/image_classification.py cpu mmdeploy_models/mmcls/fruit30_mmcls/end2end.onnx watermelon1.jpg

--image_path  --model_path  --device_name cpu


In [1]:
!python mmdeploy/demo/python/image_classification.py \
        --image_path watermelon1.jpg \
        --model_path mmdeploy_models/mmcls/fruit30_mmcls/end2end.onnx \
        --device cpu


Traceback (most recent call last):
  File "mmdeploy/demo/python/image_classification.py", line 5, in <module>
    from mmdeploy_python import Classifier
  File "/environment/miniconda3/lib/python3.7/site-packages/mmdeploy_python/__init__.py", line 21, in <module>
    from .mmdeploy_python import *  # noqa
ImportError: libonnxruntime.so.1.8.1: cannot open shared object file: No such file or directory
